In [1]:
import pandas as pd
import sqlite3

## Создаем подключение к базе данных

In [2]:
connection = sqlite3.connect('data/checking-logs.sqlite')

## Создать новую таблицу datamart в базе данных
- таблица должна иметь следующие столбцы: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts — это просто новое имя столбца timestamp из таблицы проверки, оно показывает первый коммит из конкретной лаборатории и от конкретного пользователя.
- first_view_ts — первый визит пользователя в таблицу pageviews, временная метка, когда пользователь посетил ленту новостей
- status = 'ready' все еще должен быть фильтром
- numTrials = 1 все равно должен быть фильтром
- имена labname должны быть из списка: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'
- таблица должна содержать только пользователей (uid с user_*), а не администраторов
- first_commit_ts и first_view_ts следует анализировать как datetime64[ns]

In [9]:
create_datamart_query = '''
CREATE TABLE datamart AS
SELECT
    c.uid,
    c.labname,
    DATETIME(MIN(c.timestamp)) AS first_commit_ts,
    CASE 
        WHEN MIN(p.datetime) IS NULL THEN NULL
        ELSE DATETIME(MIN(p.datetime))
    END AS first_view_ts
FROM checker c
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE
    c.status = 'ready'
    AND c.numTrials = 1
    AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname;
'''
connection.execute(create_datamart_query)
datamart_query = 'SELECT * FROM datamart'
df_datamart = pd.io.sql.read_sql(datamart_query, connection)

df_datamart['first_commit_ts'] = pd.to_datetime(df_datamart['first_commit_ts'])
df_datamart['first_view_ts'] = pd.to_datetime(df_datamart['first_view_ts'])
df_datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18,2020-04-26 21:53:59
1,user_1,laba04s,2020-04-26 17:12:11,2020-04-26 21:53:59
2,user_1,laba05,2020-05-02 19:15:18,2020-04-26 21:53:59
3,user_1,laba06,2020-05-17 16:26:35,2020-04-26 21:53:59
4,user_1,laba06s,2020-05-20 12:23:37,2020-04-26 21:53:59
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35,NaT
136,user_8,laba05,2020-05-02 13:28:07,NaT
137,user_8,laba06,2020-05-16 17:56:15,NaT
138,user_8,laba06s,2020-05-16 20:01:07,NaT


## Используя методы Pandas, создайте два фрейма данных: тестовый и контрольный
- тестовый должен иметь пользователей, которые имеют значения в first_view_ts
- контрольный должен иметь пользователей, у которых отсутствуют значения в first_view_ts
- замените отсутствующие значения в контрольном средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для будущего анализа
- сохраните обе таблицы в базе данных

In [10]:
df_test = df_datamart[df_datamart['first_view_ts'].notna()]
df_control = df_datamart[df_datamart['first_view_ts'].isna()]

average_first_view_ts = df_test['first_view_ts'].mean()

df_control.loc[:, 'first_view_ts'] = average_first_view_ts

df_test.to_sql('test', connection, if_exists='replace', index=False)
df_control.to_sql('control', connection, if_exists='replace', index=False)
print(df_control)
print(df_test)

         uid   labname     first_commit_ts                 first_view_ts
12   user_11    laba05 2020-05-03 21:06:55 2020-04-27 00:40:05.322033664
13   user_11  project1 2020-05-03 23:45:33 2020-04-27 00:40:05.322033664
14   user_12    laba04 2020-04-18 17:07:51 2020-04-27 00:40:05.322033664
15   user_12   laba04s 2020-04-26 15:42:38 2020-04-27 00:40:05.322033664
16   user_12    laba05 2020-05-03 08:39:25 2020-04-27 00:40:05.322033664
..       ...       ...                 ...                           ...
135   user_8   laba04s 2020-04-19 10:22:35 2020-04-27 00:40:05.322033664
136   user_8    laba05 2020-05-02 13:28:07 2020-04-27 00:40:05.322033664
137   user_8    laba06 2020-05-16 17:56:15 2020-04-27 00:40:05.322033664
138   user_8   laba06s 2020-05-16 20:01:07 2020-04-27 00:40:05.322033664
139   user_8  project1 2020-05-14 15:42:04 2020-04-27 00:40:05.322033664

[81 rows x 4 columns]
         uid   labname     first_commit_ts       first_view_ts
0     user_1    laba04 2020-04-26 17:0

## Закрыть соединение

In [11]:
connection.close()